# LlamaIndex re-ranker and Elasticsearch re-ranker: Comparison review

This notebook demonstrates how to use AutoGen with Elasticsearch. This notebook is based on the article [LlamaIndex re-ranker and Elasticsearch re-ranker: Comparison review](https://www.elastic.co/search-labs/blog/llamaIndex-reranker-and-elasticsearch-reranker-comparison-review).

## Installing dependencies and importing packages

In [ ]:
%pip install elasticsearch==8.17 llama-index-core llama-index-llms-openai rank-llm llama-index-postprocessor-rankgpt-rerank llama-index-embeddings-openai

In [ ]:
import json
import os
import nest_asyncio
from getpass import getpass


from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

from llama_index.core import Document, VectorStoreIndex, QueryBundle, Settings
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.postprocessor.rankgpt_rerank import RankGPTRerank
from llama_index.llms.openai import OpenAI


nest_asyncio.apply()

## Declaring variables

In [ ]:
os.environ["ELASTICSEARCH_ENDPOINT"] = getpass("Elastic Endpoint: ")
os.environ["ELASTICSEARCH_API_KEY"] = getpass("Elastic Api Key: ")
os.environ["OPENAI_API_KEY"] = getpass("OpenAI Key: ")

INDEX_NAME = "products-laptops"
INFERENCE_RERANK_NAME = "my-elastic-rerank"

### Instance a Elasticsearch client

In [ ]:
# Elasticsearch client
_client = Elasticsearch(
    os.environ["ELASTICSEARCH_ENDPOINT"],
    api_key=os.environ["ELASTICSEARCH_API_KEY"],
)

## Defining dataset

In [ ]:
products = [
    {
        "name": "ASUS ROG Strix G16",
        "description": "Powerful gaming laptop with Intel Core i9 and RTX 4070.",
        "price": 1899.99,
        "reviews": 4.7,
        "sales": 320,
        "features": [
            "Intel Core i9",
            "RTX 4070",
            "16GB RAM",
            "512GB SSD",
            "165Hz Display",
        ],
    },
    {
        "name": "Razer Blade 15",
        "description": "Premium gaming laptop with an ultra-slim design and high refresh rate.",
        "price": 2499.99,
        "reviews": 4.6,
        "sales": 290,
        "features": [
            "Intel Core i7",
            "RTX 4060",
            "16GB RAM",
            "1TB SSD",
            "240Hz Display",
        ],
    },
    {
        "name": "Acer Predator Helios 300",
        "description": "Affordable yet powerful gaming laptop with RTX graphics.",
        "price": 1399.99,
        "reviews": 4.5,
        "sales": 500,
        "features": [
            "Intel Core i7",
            "RTX 3060",
            "16GB RAM",
            "512GB SSD",
            "144Hz Display",
        ],
    },
    {
        "name": "MSI Stealth 17",
        "description": "High-performance gaming laptop with a 17-inch display.",
        "price": 2799.99,
        "reviews": 4.8,
        "sales": 200,
        "features": ["Intel Core i9", "RTX 4080", "32GB RAM", "1TB SSD", "4K Display"],
    },
    {
        "name": "Dell XPS 15",
        "description": "Sleek and powerful ultrabook with a high-resolution display.",
        "price": 2199.99,
        "reviews": 4.7,
        "sales": 350,
        "features": [
            "Intel Core i7",
            "RTX 3050 Ti",
            "16GB RAM",
            "1TB SSD",
            "OLED Display",
        ],
    },
    {
        "name": "HP Omen 16",
        "description": "Gaming laptop with a balanced price-to-performance ratio.",
        "price": 1599.99,
        "reviews": 4.4,
        "sales": 280,
        "features": [
            "AMD Ryzen 7",
            "RTX 3060",
            "16GB RAM",
            "512GB SSD",
            "165Hz Display",
        ],
    },
    {
        "name": "Lenovo Legion 5 Pro",
        "description": "Powerful Ryzen-powered gaming laptop with high refresh rate.",
        "price": 1799.99,
        "reviews": 4.6,
        "sales": 400,
        "features": [
            "AMD Ryzen 9",
            "RTX 3070 Ti",
            "16GB RAM",
            "1TB SSD",
            "165Hz Display",
        ],
    },
    {
        "name": "MacBook Pro 16",
        "description": "Apple's most powerful laptop with M3 Max chip.",
        "price": 3499.99,
        "reviews": 4.9,
        "sales": 500,
        "features": [
            "Apple M3 Max",
            "32GB RAM",
            "1TB SSD",
            "Liquid Retina XDR Display",
        ],
    },
    {
        "name": "Alienware m18",
        "description": "High-end gaming laptop with extreme performance.",
        "price": 2999.99,
        "reviews": 4.8,
        "sales": 150,
        "features": [
            "Intel Core i9",
            "RTX 4090",
            "32GB RAM",
            "2TB SSD",
            "480Hz Display",
        ],
    },
    {
        "name": "Samsung Galaxy Book3 Ultra",
        "description": "Ultra-lightweight yet powerful laptop with AMOLED display.",
        "price": 2099.99,
        "reviews": 4.5,
        "sales": 180,
        "features": [
            "Intel Core i7",
            "RTX 4070",
            "16GB RAM",
            "512GB SSD",
            "AMOLED Display",
        ],
    },
    {
        "name": "Microsoft Surface Laptop 5",
        "description": "Sleek productivity laptop with great battery life.",
        "price": 1699.99,
        "reviews": 4.3,
        "sales": 220,
        "features": ["Intel Core i7", "16GB RAM", "512GB SSD", "Touchscreen"],
    },
    {
        "name": "Gigabyte AORUS 17",
        "description": "Performance-focused gaming laptop with powerful cooling.",
        "price": 1999.99,
        "reviews": 4.6,
        "sales": 250,
        "features": [
            "Intel Core i9",
            "RTX 4070",
            "16GB RAM",
            "1TB SSD",
            "360Hz Display",
        ],
    },
]

## Indexing documents to Elasticsearch

In [ ]:
# Creating mapping for the index
try:
    _client.indices.create(
        index=INDEX_NAME,
        body={
            "mappings": {
                "properties": {
                    "name": {"type": "text", "copy_to": "semantic_field"},
                    "description": {"type": "text", "copy_to": "semantic_field"},
                    "price": {"type": "float", "copy_to": "semantic_field"},
                    "reviews": {"type": "float", "copy_to": "semantic_field"},
                    "sales": {"type": "integer", "copy_to": "semantic_field"},
                    "features": {"type": "keyword", "copy_to": "semantic_field"},
                    "semantic_field": {"type": "semantic_text"},
                }
            }
        },
    )

    print("index created successfully")
except Exception as e:
    print(
        f"Error creating inference endpoint: {e.info['error']['root_cause'][0]['reason'] }"
    )

In [ ]:
# Creating inference endpoint for re-ranking
try:
    _client.options(
        request_timeout=60, max_retries=3, retry_on_timeout=True
    ).inference.put(
        task_type="rerank",
        inference_id=INFERENCE_RERANK_NAME,
        body={
            "service": "elasticsearch",
            "service_settings": {
                "model_id": ".rerank-v1",
                "num_threads": 1,
                "adaptive_allocations": {
                    "enabled": True,
                    "min_number_of_allocations": 1,
                    "max_number_of_allocations": 4,
                },
            },
        },
    )

    print("Inference endpoint created successfully.")

except Exception as e:
    print(
        f"Error creating inference endpoint: {e.info['error']['root_cause'][0]['reason'] }"
    )

In [ ]:
def build_data():
    for doc in products:
        yield {"_index": INDEX_NAME, "_source": doc}


try:
    success, errors = bulk(_client, build_data())
    print(f"{success} documents indexed successfully")

    if errors:
        print("Errors during indexing:", errors)

except Exception as e:
    print(f"Error: {str(e)}")

## LlamaIndex re-rankers

In [ ]:
rerank_promt = "Best laptops for developers and designers"

### documents formating

In [ ]:
document_objects = []

for doc in products:
    text_content = f"""
      Product Name: {doc["name"]}
      Description: {doc["description"]}
      Price: ${doc["price"]}
      Reviews: {doc["reviews"]} stars
      Sales: {doc["sales"]} units sold
      Features: {', '.join(doc["features"])}
      """

    document_objects.append(Document(text=text_content))

index = VectorStoreIndex.from_documents(document_objects)

### LLM endpoint

In [ ]:
Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.chunk_size = 512

In [ ]:
index = VectorStoreIndex.from_documents(
    document_objects,
)

In [ ]:
# Re-rank method
def get_retrieved_nodes(
    query_str, vector_top_k=10, reranker_top_n=3, with_reranker=False
):
    query_bundle = QueryBundle(query_str)
    # configure retriever
    retriever = VectorIndexRetriever(
        index=index,
        similarity_top_k=vector_top_k,
    )
    retrieved_nodes = retriever.retrieve(query_bundle)

    if with_reranker:
        # configure re-ranker
        reranker = RankGPTRerank(
            llm=OpenAI(
                model="gpt-3.5-turbo-16k",
                temperature=0.0,
                api_key=os.environ["OPENAI_API_KEY"],
            ),
            top_n=reranker_top_n,
            verbose=True,
        )
        retrieved_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)

    return retrieved_nodes

In [ ]:
def visualize_retrieved_nodes(nodes):
    result_dicts = []

    for node in nodes:
        result_dict = {"Score": node.score, "Text": node.node.get_text()}
        result_dicts.append(result_dict)

    return result_dicts

### Without re-rank

In [ ]:
new_nodes = get_retrieved_nodes(
    rerank_promt,
    vector_top_k=12,
    with_reranker=False,
)

results = visualize_retrieved_nodes(new_nodes[:3])

print(f"Top 3 results without reranking: {json.dumps(results, indent=2)}")

### With re-rank

In [ ]:
new_nodes = get_retrieved_nodes(
    rerank_promt,
    vector_top_k=12,
    reranker_top_n=3,
    with_reranker=True,
)

results = visualize_retrieved_nodes(new_nodes[:3])
print(f"Top 3 results with reranking: {json.dumps(results, indent=2)}")

## Elasticsearch re-rankers

In [ ]:
async def es_search(query):
    response = _client.search(index=INDEX_NAME, body=query)
    hits = response["hits"]["hits"]

    if not hits:
        return ""

    return json.dumps([hit["_source"] for hit in hits], indent=2)

In [ ]:
# Query with semantic search
result = await es_search(
    {
        "size": 3,
        "query": {
            "semantic": {
                "field": "semantic_field",
                "query": rerank_promt,
            }
        },
        "_source": {
            "includes": [
                "name",
                "description",
                "price",
                "reviews",
                "sales",
                "features",
            ]
        },
    }
)

print(f"Query result: {result}")

In [ ]:
results = await es_search(
    {
        "size": 3,
        "_source": {
            "includes": [
                "name",
                "description",
                "price",
                "reviews",
                "sales",
                "features",
            ]
        },
        "retriever": {
            "text_similarity_reranker": {
                "retriever": {
                    "standard": {
                        "query": {
                            "semantic": {
                                "field": "semantic_field",
                                "query": rerank_promt,
                            }
                        }
                    }
                },
                "field": "text",
                "inference_id": INFERENCE_RERANK_NAME,
                "inference_text": rerank_promt,
                "rank_window_size": 10,
                "min_score": 0.5,
            }
        },
    }
)
print(f"Query result: {result}")

## Cleaning environment

Delete the resources used to prevent them from consuming resources.

In [ ]:
def print_results(results):
    if results.get("acknowledged", False):
        print("DELETED successfully.")

    if "error" in results:
        print(f"ERROR: {results['error']['root_cause'][0]['reason']}")


# Cleanup - Delete Index
result = _client.indices.delete(index=INDEX_NAME, ignore=[400, 404])
print_results(result)


# Cleanup - Inference endpoint
result = _client.inference.delete(inference_id=INFERENCE_RERANK_NAME, ignore=[400, 404])
print_results(result)